
# Transfert d'organe sous incertitude sur la compatibilité

Sibylline Moukarzel, Matthieu Roux

## Introduction

Malgré l'augmentation croissante du nombre de transplantations d'organes effectuées chaque année (environ 6000 en 2017 dont 3782 transplantations de reins), la demande reste en perpétuelle augmentation. Ainsi 6000 organes, dont 3782 reins, ont été transplantés en 2017, mais il y avait encore 24000 personnes en attente d'un organe la même année. Les organes transplantés peuvent provenir d'un donneur décédé ou, dans le cas des reins et du foie, d'un donneur vivant consentant, le plus souvent membre de la famille du patient. Hélas, même si un proche accepte de prendre ce risque pour sa santé, il ne sera pas forcément compatible avec le patient. Pour cette raison, les pratiques médicales les législations évoluent dans de nombreux pays afin de permettre la mise en place d'un programme d'échange de dons d'organes.

L'exemple le plus simple d'échange de don d'organes est celui où deux patients $P_1$ et $P_2$ sont accompagnés de donneurs $D_1$ et $D_2$. Les patients sont supposés incompatibles avec les donneurs qui les accompagnent, mais on suppose que $D_1$ est compatible avec $P_2$ et $D_2$ avec $P_1$. Il est alors possible de transplanter un organe de $D_1$ vers $P_2$ et de $D_2$ vers $P_1$ avec le consentement de tous et en suivant la procédure légale.

Plus généralement, un cycle d'échange d'organes associe $k$ paires de patient-donneur $(P_{i_1},D_{i_1}), \dots, P_{i_k},D_{i_k})$ de sorte que $D_{i_l}$ donne à $P_{i_{l+1}}$ pour $l=1,\dots,k-1$ et $D_{i_k}$ donne à $P_{i_1}$.
Par ailleurs, le point essentiel est que les transferts soient tous réalisés en même temps et dans le même hôpital pour éviter qu'une rétractation de dernière minute ne lèse un patient et son donneur, et que les patients et donneurs venus ensemble et leur famille puissent se soutenir émotionnellement durant l'hospitalisation. 
Pour cette raison, le nombre d'échanges prenant place au sein d'un même cycle est nécessairement limité. En pratique, l'organisation d'un cycle de trois paires est déjà une épreuve pour le personnel d'un hôpital, et le plus grand cycle ayant jamais eu lieu a a impliqué six patients et donneurs.

Dans ce projet, nous prendrons le point de vue de l'organisme national responsable de la gestion du programme d'échange d'organes. 
À chaque phase d'échange, l'objectif de cet organisme est de choisir un ensemble de cycles d'échanges entre paires compatibles afin de maximiser le nombre de patients recevant un organe. Dans certains cas, on peut aussi donner une priorité à certains patients en fonction de la gravité de leur état ou de la durée de leur attente. 
Pour cela, on pourra attribuer des poids différents à chaque patient et maximiser la somme des poids des patients recevant un organe. 

Lors d'une première phase de planification, l'organisme ne dispose que de données individuelles sur chaque donneur et chaque receveur pour déduire la compatibilité entre donneurs et patients. 
Ces données sont principalement le groupe sanguin et le complexe majeur d’histocompatibilité, aussi appelé système HLA. 
Ils en tirent un premier graphe de compatibilité orienté, $G=(V,A)$, où chaque sommet de $V$ représente une paire donneur-patient et où un arc entre deux paires $(P_k,D_k)$ et $(P_l,D_l)$ signifie que $D_k$ est __a priori__ compatible avec $P_l$.
Cependant, la compatibilité effective entre deux personnes ne peut être assurée qu'en mettant en présence des tissus des deux personnes dans ce que l'on appelle un _test croisé_. 
En général, on peut supposer que les données individuelles permettent de déterminer une probabilité de réussite du test croisé.
Mais, dans tous les cas, ces tests peuvent être lourds à réaliser pour les patients et demander des ressources importantes aupèrs des services hospitaliers, donc leur nombre sera toujours limité. 
On pourra pour cela considérer une limite fixe, une limite dépendant du nombre de paires patient-donneur ou bien supposer que les tests ne servent qu'à confirmer la compatibilité après avoir décidé les cycles d'échange entre patients a priori compatibles. 

## Description des données

Des jeux de données correspondant à des ensembles de paires patient-donneur ont été partagés dans la PrefLib (https://www.preflib.org/data/MD). Le sous-ensemble d'instances auxquels vous pourrez vous intéresser dans un premier temps accompagne ce sujet sur Moodle. Les dix premières instances (numérotées de 1 à 10) contiennent 10 paires patient-donneur, les 10 suivantes (numérotées de 31 à 40) en contiennent 32 et les 10 dernières (numérotées de 71 à 80) en contiennent 64. Chaque jeu de données est décrit par deux fichiers, l'un énumérant les données relatives à chaque paire et portant l'extension .dat, et l'autre énumérant les données relatives aux arcs et portant l'extension .wmd.
Nous vous fournissons une fonction permettant de lire les fichiers relatifs à un jeu de donnéees. 

In [3]:
# include all the necessary packages, if some of them are not installed, you will need to install them before
using Random, MetaGraphs, SimpleWeightedGraphs,JuMP, DelimitedFiles, Distributions, NBInclude, GLPK, Gurobi, GraphPlot

In [2]:
# Inclusion du fichier contenant la fonction de parsing des données
@nbinclude("dataparser.ipynb")
# Exemple d'utilisation de la fonction supposant que les fichiers sont dans le dossier data
pref_path = "C:/Users/matth/Documents/5GM/optim sous incertitude/stochastic_optimization/data/"

kep_graph = read_kep_file(pref_path*"MD-00001-00000001.wmd",pref_path*"MD-00001-00000001.dat");

Une fois le graphe de compatibilité donné, une instance est entièrement décrite par la connaissance de la distribution des incertitudes dans une approche par optimisation stochastique. Dans une approche par optimisation robuste, le pire cas est déjà connu pour chaque arête, il s'agit d'un échec de la transplantation. Plusieurs modèles d'incertitudes sont classiquement regardés dans la littérature, mais tous considèrent que la réussite du test croisé réalisé sur un arc $a$ suit une loi de Bernouilli de probabilité $1-f_a$ où $f_a$ est une probabilité d'échec donnée. Nous donnons ci-dessous la fonction permettant de calculer des probabilités d'échec pour tous les arcs en fonction d'un paramètre à choisir dans le tableau DISTRIBUTIONS.

In [4]:
DISTRIBUTIONS = ["Constant","Binomial","BinomialUNOS","BinomialAPD","NoFailure"]

"""
    get_failure_rates

Generate failure rates on each edge, and add its value as a property to the edge of the kep_graph. 

# Parameters
* `kep_graph::MetaDiGraph` : graph describing the pairs and compatibilities
* `distribution::String` : type of distirbution of uncertainties; to be chosen in the DISTRIBUTIONS vector
"""
function get_failure_rates(kep_graph::MetaDiGraph, distribution::String)

    failure_rates = []

    for edge in MetaGraphs.edges(kep_graph)
        # Failure rates depend on the chosen distribution of uncertainties
        if distribution == "Constant"
            # constant failure rates equal to 70%
            set_prop!(kep_graph, edge, :failure, 0.7)
        elseif distribution == "Binomial"
            if rand() < 0.25
                # random failure rates equal to 10% on average for 25% edges
                set_prop!(kep_graph, edge, :failure, rand() * 0.2)
            else
                # random failure rates equal to 90% on average for 75% edges
                set_prop!(kep_graph, edge, :failure, 0.8 + rand() * 0.2)
            end
        elseif distribution == "BinomialUNOS"
            # %pra denotes the panel reactive antibody level
            # %pra of the patient < 0.8 : UNOS low sensitized patients
            if get_prop(kep_graph, edge.dst, :pra) < 0.8
                # failure rate equal to 10% if the patient is low sensitized
                set_prop!(kep_graph, edge, :failure, 0.1)
            else
                # failure rate equal to 90% otherwise 
                set_prop!(kep_graph, edge, :failure, 0.9)
            end
        elseif distribution  == "BinomialAPD"
            # %pra denotes the panel reactive antibody level
            # %pra of the patient < 0.75 : APD low sensitized patients
            if get_prop(kep_graph, edge.dst, :pra) < 0.75
                # failure rate equal to 28% if the patient is low sensitized
                set_prop!(kep_graph, edge, :failure, 0.28)
            else
                # failure rate equal to 58% otherwise 
                set_prop!(kep_graph, edge, :failure, 0.58)
            end
        elseif distribution == "NoFailure"
            # failure rates equal to 0
            set_prop!(kep_graph, edge, :failure, 0.)
        end
    end
end


get_failure_rates

In [5]:
failure_rates = get_failure_rates(kep_graph, "Binomial")

In [6]:
print([get_prop(kep_graph,e,:failure) for e in MetaGraphs.edges(kep_graph)])

[0.8046513849026091, 0.9899641428387606, 0.8083712754963126, 0.9629117631578932, 0.9520090791674324, 0.8061975691914377, 0.9157068104959708, 0.9533448123939465, 0.9117097937116788, 0.9483886879129709, 0.8128834181425083, 0.9934956771089628, 0.029937928242653955, 0.9036249942059066, 0.9302844716649261, 0.9377414707477485, 0.9945135443042137, 0.02931873521570305, 0.8954204606912158, 0.12383442277759227, 0.9112982637506051, 0.12221449200582409, 0.9242269777110232, 0.9048946272559862, 0.15816354291681123, 0.9560897763873641, 0.9780095474986592, 0.9245153652394323, 0.9915457049486893, 0.18075124140911297, 0.9093324480628634, 0.8140296854570945, 0.9439945275517125, 0.05051509761592854, 0.07805766402483616, 0.8105449993947477, 0.9563846438245569, 0.19848492682132401, 0.941741879226232, 0.8460130966810233, 0.8355944426443067, 0.968243978230412, 0.8291867850355547, 0.8312891823014106, 0.8351063723315516, 0.9095701940772452, 0.9961955134089304, 0.9679231609297102, 0.9234100496908134, 0.897216068

## Travail à réaliser

Nous vous donnons une grande liberté sur la façon de traiter le sujet. En fonction de décisions que vous justifierez, vous pourrez traiter le problème par une approche d'optimisation stochastique, d'optimisation robuste ou de toute autre approche averse aux risques. Le travail commencera par décrire l'approche suivie puis le modèle en découlant. Un code Julia permettra ensuite d'implémenter une ou plusieurs méthodes de résolution pour le modèle. Vous pourrez tester la ou les méthodes sur des instances de la PrefLib. Vos interprétations devront rendre compte des enjeux pratiques et des enjeux algorithmiques (optimalité, temps de calcul, passage à l'échelle, etc.) de votre travail.
Le résultat de votre travail sera à rendre dans ce notebook avant le 14 janvier 2022. Chaque cellule du notebook aura préalablement été exécutée (sans erreur, évidemment), et il importera que les affichages utilisés dans vos interprétations y apparaissent. 

In [8]:
x = [0, 10, 20, 30, 40, 40, 40, 40, 40, 30, 20, 10, 0, 0, 0, 0]
y = [0, 0, 0, 0, 0, 10, 20, 30, 40, 40, 40, 40, 40, 30, 20, 10]
nodelabel = [get_prop(kep_graph,v,:pra) for v in MetaGraphs.vertices(kep_graph)]


16-element Vector{Float64}:
 0.05
 0.05
 0.05
 0.5875
 0.05
 0.45
 0.45
 0.45
 0.45
 0.2875
 0.05
 0.5875
 0.925
 0.45
 0.05
 0.2875

In [11]:
for v in MetaGraphs.vertices(kep_graph)
    println(get_prop(kep_graph,v,:pra),"\n")
end

0.05

0.05

0.05

0.5875

0.05

0.45

0.45

0.45

0.45

0.2875

0.05

0.5875

0.925

0.45

0.05

0.2875



## Modélisation déterministe
On modélise d'abord notre problème en supposant que la probabilité d'échange pour chaque arc est de 1. On considère également qu'il n'y a pas de limite à la taille des cycles d'échanges.

On note $w_{i,j} = 1, ~ \forall (i,j) \in A$, les poids des arcs du graphe.

On considère les variables suivantes :
- $x_{i,j} \in \{1,0\}$, $x_{i,j} = 1$  si le donneur de la paire i donne au patient de la paire j 


Objectif :

maximiser le nombre d'échange : max $\sum_{i,j : (i,j) \in A} x_{ij} w_{ij}$   

Contraintes :

- $\sum_{j : (j,i) \in A} x_{ji} = \sum_{j : (i,j) \in A} x_{ij}$, $\forall i \in V$, (le patient i recoit un rein ssi le donneur i donne un rein)

- $\sum_{j : (i,j) \in A} x_{ij} \le 1$, $\forall i \in V$, ( un donneur peut donner seulement un rein) 

In [ ]:
function first_MIP(kep_graph)
    #modele = Model(Gurobi.Optimizer)
    modele = Model(GLPK.Optimizer)
    
    # Nombre de paires de donneurs/patients, nombre de noeuds du graphe :
    P = MetaGraphs.nv(kep_graph)
    #N
    compatible = [(i,j) for i in 1:nb_vertices, j in 1:nb_vertices if MetaGraphs.has_edge(kep_graph, i,j)]

    # Variable : indique si le patient de la paire j a un rein du donneur de la paire i
@variable(modele,x[i in 1:P,j in 1:P],Bin)
    
# Contrainte : le patient i recoit un rein ssi le donneur i donne un rein
@constraint(modele,recoit[i in 1:P],sum(x[j,i] for j in 1:P if (i,j) in compatible) == sum(x[i,j] for j in 1:P if ((i,j) in compatible)))
    # Contrainte : un donneur peut donner seulement un rein
@constraint(modele,capacite[i in 1:P],sum(x[i,j] for j in 1:P if (i,j) in compatible)<=1)
    # Contrainte : on veut des cycles plus petits que L
    
    # Objectif : maximiser le nombre total de transplantations
@objective(modele,Max,sum(edges_weight[i,j]*x[i,j] for i in 1:P,j in 1:P if (i,j) in compatible))
    
set_optimizer_attribute(modele, MOI.Silent(), true)
optimize!(modele)

return JuMP.objective_value(modele),JuMP.value.(x)
end

In [36]:
edges_weight = zeros(MetaGraphs.nv(kep_graph), MetaGraphs.nv(kep_graph))
for (i,j) in compatible 
    edges_weight[i,j] = 1
end

(14.0, [0.0 1.0 … 1.0 0.0; 1.0 0.0 … 0.0 0.0; … ; 1.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0])